# Modeling

In [59]:
#Train and Test Set
X_train, X_test, y_train, y_test = train_test_split(df_ready, training_labels,test_size=0.2)

First define loss based on 5 Croos Validation

In [60]:
n_folds = 5

# squared_loss
def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True).get_n_splits(numerical_features)
    return cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kf)

def rmse_lv_cv(model):
    kf = KFold(n_folds, shuffle=True).get_n_splits(numerical_features)
    return cross_val_score(model, Xlv_train, y_train, scoring='neg_mean_squared_error', cv=kf)

## Model Exploration

In [61]:
for Model in [LinearRegression, Ridge, Lasso, ElasticNet, RandomForestRegressor, XGBRegressor, HuberRegressor]:
    if Model == XGBRegressor: cv_res = rmse_cv(XGBRegressor(objective='reg:squarederror'))
    else: cv_res = rmse_cv(Model())
    print('{}: {:.5f} +/- {:5f}'.format(Model.__name__, -cv_res.mean(), cv_res.std()))

LinearRegression: 35333.16517 +/- 8182.276043
Ridge: 35328.73834 +/- 8184.273997
Lasso: 36048.76825 +/- 8250.358356
ElasticNet: 37859.71859 +/- 8447.857286
RandomForestRegressor: 34722.18731 +/- 6454.725846
XGBRegressor: 36321.14997 +/- 6127.005616
HuberRegressor: 37022.75564 +/- 8435.616434


In [62]:
def evaluation(y_test, y_predict):  
    mae = metrics.mean_absolute_error(y_test, y_predict)
    mse = metrics.mean_squared_error(y_test, y_predict)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_predict))
    r2_square = metrics.r2_score(y_test, y_predict)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)

In [100]:
def evaluation(y_test, y_predict,mode,model):  
    mae = metrics.mean_absolute_error(y_test, y_predict)
    mse = metrics.mean_squared_error(y_test, y_predict)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_predict))
    r2_square = metrics.r2_score(y_test, y_predict)
    #print('MAE:', mae)
    #print('MSE:', mse)
    #print('RMSE:', rmse)
    #print('R2 Square', r2_square)
    result=pd.DataFrame({'Algorithm':[model],
            mode+'_MAE': [mae],
            mode+'_MSE': [mse],
            mode+'_RMSE': [rmse],
            mode+'_R2 Square': [r2_square],
             })
    return result

## Hyperparameter tuning

### Linear Regression

In [107]:
### Linear Regression ###

def linear_reg(input_x, input_y, cv=5):
    ## Defining parameters
    global grid_search_LR
    model_LR= LinearRegression()

    parameters = {'fit_intercept':[True,False], 'copy_X':[True, False]}

    ## Building Grid Search algorithm with cross-validation and Mean Squared Error score.

    grid_search_LR = GridSearchCV(estimator=model_LR,  
                         param_grid=parameters,
                         scoring='neg_mean_squared_error',
                         cv=cv,
                         n_jobs=-1)

    ## Lastly, finding the best parameters.

    grid_search_LR.fit(input_x, input_y)
    return grid_search_LR



# Train
linear_reg(X_train, y_train)
best_parameters_LR = grid_search_LR.best_params_  
best_score_LR = grid_search_LR.best_score_ 
print(best_parameters_LR)
final_model_LR = grid_search_LR.best_estimator_
y_predict_LR = final_model_LR.predict(X_train)
evaluation(y_train, y_predict_LR,mode='Training',model='Linear Regression')

{'copy_X': True, 'fit_intercept': False}


,Algorithm,Training_MAE,Training_MSE,Training_RMSE,Training_R2 Square
0,Linear Regression,68.937626,35081.858903,187.301519,0.165472


In [108]:
#Test Data
y_predict_test_LR = final_model_LR.predict(X_test)
evaluation(y_test, y_predict_test_LR,mode='Test',model='Linear Regression')

,Algorithm,Test_MAE,Test_MSE,Test_RMSE,Test_R2 Square
0,Linear Regression,70.855571,40891.929071,202.217529,0.137353


### Ridge Regression

In [120]:
### Ridge Regression ###

def ridge_reg(input_x, input_y, cv=5):
    ## Defining parameters
    global grid_search_Ridge
    model_Ridge= Ridge()

    # prepare a range of alpha values to test
    alphas =np.arange(20,31)#np.array([30,25,20])

    ## Building Grid Search algorithm with cross-validation and Mean Squared Error score.

    grid_search_Ridge = GridSearchCV(estimator=model_Ridge,  
                         param_grid=(dict(alpha=alphas)),
                         scoring='neg_mean_squared_error',
                         cv=cv,
                         n_jobs=-1)

    ## Lastly, finding the best parameters.

    grid_search_Ridge.fit(input_x, input_y)
    return grid_search_Ridge



# Train
ridge_reg(X_train, y_train)
best_parameters_Ridge = grid_search_Ridge.best_params_  
best_score_Ridge = grid_search_Ridge.best_score_ 
print(best_parameters_Ridge)
final_model_Ridge = grid_search_Ridge.best_estimator_
y_predict_Ridge = final_model_Ridge.predict(X_train)
evaluation(y_train, y_predict_Ridge,mode='Training',model='Ridge Regression')

{'alpha': 24}


,Algorithm,Training_MAE,Training_MSE,Training_RMSE,Training_R2 Square
0,Ridge Regression,68.778807,35098.271236,187.345326,0.165082


In [118]:
#Test Data
y_predict_test_Ridge = final_model_Ridge.predict(X_test)
evaluation(y_test, y_predict_test_Ridge,mode='Test',model='Ridge Regression')

,Algorithm,Test_MAE,Test_MSE,Test_RMSE,Test_R2 Square
0,Ridge Regression,70.648309,40888.00506,202.207826,0.137436


### Random Forest

In [121]:
### Random Forest ###

def rf_reg(input_x, input_y, cv=5):
    ## Defining parameters
    global grid_search_rf
    model_rf= RandomForestRegressor()

    param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]
    

    ## Building Grid Search algorithm with cross-validation and Mean Squared Error score.

    grid_search_rf = GridSearchCV(model_rf, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

    ## Lastly, finding the best parameters.

    grid_search_rf.fit(input_x, input_y)
    return grid_search_rf



# Train
rf_reg(X_train, y_train)
best_parameters_rf = grid_search_rf.best_params_  
best_score_rf = grid_search_rf.best_score_ 
print(best_parameters_rf)
final_model_rf = grid_search_rf.best_estimator_
y_predict_rf = final_model_rf.predict(X_train)
evaluation(y_train, y_predict_rf,mode='Training',model='Random Forest')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.87711e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.87711e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.87711e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.87711e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:212: LinAl

{'max_features': 8, 'n_estimators': 30}


,Algorithm,Training_MAE,Training_MSE,Training_RMSE,Training_R2 Square
0,Random Forest,27.540097,7273.924047,85.287303,0.826968


In [122]:
#Test Data
y_predict_test_rf = final_model_rf.predict(X_test)
evaluation(y_test, y_predict_test_rf,mode='Test',model='Random Forest')

,Algorithm,Test_MAE,Test_MSE,Test_RMSE,Test_R2 Square
0,Random Forest,65.233881,41902.365721,204.700673,0.116037


In [125]:
### Random Forest ###

def rf_reg(input_x, input_y, cv=5):
    ## Defining parameters
    global rnd_search_rf
    model_rf= RandomForestRegressor()

    
    param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }
    
    

    ## Building Grid Search algorithm with cross-validation and Mean Squared Error score.

    rnd_search_rf = RandomizedSearchCV(model_rf, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error')

    ## Lastly, finding the best parameters.

    rnd_search_rf.fit(input_x, input_y)
    return rnd_search_rf



# Train
rf_reg(X_train, y_train)
best_parameters_rf = rnd_search_rf.best_params_  
best_score_rf = rnd_search_rf.best_score_ 
print(best_parameters_rf)
final_model_rf = rnd_search_rf.best_estimator_
y_predict_rf = final_model_rf.predict(X_train)
evaluation(y_train, y_predict_rf,mode='Training',model='Random Forest')

{'max_features': 7, 'n_estimators': 122}


,Algorithm,Training_MAE,Training_MSE,Training_RMSE,Training_R2 Square
0,Random Forest,26.755158,6888.124341,82.994725,0.836145


In [126]:
#Test Data
y_predict_test_rf = final_model_rf.predict(X_test)
evaluation(y_test, y_predict_test_rf,mode='Test',model='Random Forest')

,Algorithm,Test_MAE,Test_MSE,Test_RMSE,Test_R2 Square
0,Random Forest,64.528012,41198.686611,202.974596,0.130882


In [127]:
rf = RandomForestRegressor()

#Number of trees in the forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop=2000,num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2,5,10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator=rf, param_distributions = random_grid, n_iter=10, cv = 3, verbose=2, random_state=42, n_jobs=-1)

rf_random.fit(X_train, y_train)

best_random = rf_random.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=  18.3s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.1min
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 6.1min
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=11.4min
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=27.8min
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=  18.5s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.1min
[CV] END bootstrap=True, max_depth=3

/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=  18.7s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 6.7min
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=12.0min
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=25.8min
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000; total time=11.8min


/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.2min
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 6.5min
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=  45.3s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=  45.6s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=  45.9s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=26.3min
[CV] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1200; total time= 9.6min
[CV] END bootstrap=True, max_depth=

/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

In [128]:
best_random

RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_split=5,
                      n_estimators=1400)

In [130]:
best_random = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=1400,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)
rfr_CV_best = -rmse_cv(best_random)
best_random.fit(X_train, y_train) 
y_train_rfr = best_random.predict(X_train)
y_test_rfr = best_random.predict(X_test)

In [131]:
evaluation(y_train, y_train_rfr,mode='Training',model='Random Forest')

,Algorithm,Training_MAE,Training_MSE,Training_RMSE,Training_R2 Square
0,Random Forest,44.560461,15052.800433,122.689855,0.641924


In [132]:
evaluation(y_test, y_test_rfr,mode='Test',model='Random Forest')

,Algorithm,Test_MAE,Test_MSE,Test_RMSE,Test_R2 Square
0,Random Forest,63.392379,39559.025456,198.894508,0.165472
